In [1]:
from tqdm import tqdm
from analysis.nlp_transformations import separate_previous_conversation, remove_footers
from typing import List
from dao.email import DAOEmailGenerated, DAORealEmail

from dao.attribute import DAOAttribute
from analysis.nlp_transformations import replace_links_with_text

/home/pawel/.virtualenvs/anti-gpt-checker/lib/python3.10/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/home/pawel/.virtualenvs/anti-gpt-checker/lib/python3.10/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
[nltk_data] Downloading package stopwords to /home/pawel/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
from config import init_spacy_polish_nlp_model, init_spacy_english_nlp_model

init_spacy_polish_nlp_model()
init_spacy_english_nlp_model()

In [3]:
from analysis.attribute_retriving import calculate_burstiness
from analysis.nlp_transformations import lemmatize_text
from models.attribute import AttributeInDB

dao_generated_emails: DAOEmailGenerated = DAOEmailGenerated()
dao_real_emails: DAORealEmail = DAORealEmail()
dao_attribute: DAOAttribute = DAOAttribute()


all_attributes: List[AttributeInDB] = dao_attribute.find_many_by_query({'burstiness': None})

for attribute in tqdm(all_attributes, total=len(all_attributes), desc=f'Calculating burstiness', unit='emails', miniters=1):
    if attribute.referenced_db_name == 'email_generated_dataset':
        og_email = dao_generated_emails.find_one_by_query({'_id': attribute.referenced_doc_id})
        language = og_email.language
    else:
        og_email = dao_real_emails.find_one(attribute.referenced_db_name, {'_id': attribute.referenced_doc_id})
        language = og_email.detected_language
    _, og_text = separate_previous_conversation(og_email.text_plain)
    og_text = remove_footers(og_text)
    og_text = replace_links_with_text(og_text, "")
    lem_text, _ = lemmatize_text(og_text, language)
    lem_text = lem_text.strip()
    burstiness = calculate_burstiness(lem_text, language)
    dao_attribute.update_one({'_id': attribute.id}, {'$set':{'burstiness': burstiness}})
    if attribute.referenced_db_name == 'email_generated_dataset':
        dao_generated_emails.update_one({'_id': attribute.referenced_doc_id}, {'$set':{'lemmatized_body': lem_text}})
    else:
        dao_real_emails.update_one(attribute.referenced_db_name, {'_id': attribute.referenced_doc_id}, {'$set':{'lemmatized_body': lem_text}})

Calculating burstiness: 100%|██████████| 12561/12561 [33:17<00:00,  6.29emails/s]  
